# Automatiseer repetitieve taken

Notebook ontwikkeld voor de belevingsdagen op Thomas More Mechelen.<br>
Bron: https://automatetheboringstuff.com/



In [ ]:
! pip install requests
! pip install bs4

## Webscraping
Scrape alle foto's van een website

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import os

url = "<url naar pagina>"
folder = "./images/"

# als map niet bestaat, maak aan
if not os.path.exists(folder):
    os.makedirs("images")

# Maak een verzoek naar de website en verkrijg de inhoud als een BeautifulSoup object.
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Loop over alle img-tags
for img_tag in soup.find_all("img"):
    # Vraag de source op van de afbeelding
    img_src = img_tag.get("src")

    # Analyseer de bron-URL om de bestandsnaam te verkrijgen.
    filename = os.path.basename(urlparse(img_src).path)

    # Download de afbeelding
    with open(os.path.join(folder, filename), "wb") as f:
        img_url = urljoin(url, img_src)
        response = requests.get(img_url)
        f.write(response.content)
        print("Gedownload", filename)


## Werken met bestanden & mappen
Verwijder alle niet -jpg bestanden

In [ ]:
import os
import fnmatch as fm

folder = "./images/"

# Loop over alle bestanden in de folder
for file in os.listdir(folder):
    
    # Controleer of het bestand geen .jpg-extensie heeft.
    if not fm.fnmatch(file, "*.jpg"):
        # Zoniet, verwijder bestand!
        os.remove(os.path.join(folder, file))
        print("Verwijderd",file)


## Werken met API’s
Verwijder achtergrond van foto, we werken met de API https://clipdrop.co/apis/docs/remove-background<br>
Vraag eerst een key aan, je krijgt 100 credits om mee te starten.

In [ ]:
import requests

# Stel de API-eindpunt URL in.
url = 'https://clipdrop-api.co/remove-background/v1'

# Stel het pad naar uw afbeeldingsbestand in.
img_path = 'images/jump.jpg'

# Stel de API-key in de headers in.
headers = {
    'x-api-key': '<Je eigen API-key>'
}

# Open het afbeeldingsbestand en lees de binair inhoud.
with open(img_path, 'rb') as img_file:
    img_object = img_file.read()

# Stel de files-parameter in als een dictionary met de binair afbeeldingsinhoud.
files = {
    'image_file': ('image.jpg', img_object, 'image/jpeg')
}

# Stuur POST request naar ClipDrop API
response = requests.post(url, headers=headers, files=files)

# Check of je een ok antwoord terug kreeg (status code 200)
if response.ok:
    # Bewaar transparante afbeelding
    with open('images/transparent.png', 'wb') as new_img:
        new_img.write(response.content)
else:
    print("Er was een probleem!")



# E-mailen

Gebruik de SMTP-server van Google om e-mails te versturen. Stel 2 stapsverificatie in en maak een app-wachtwoord: https://support.google.com/mail/answer/185833?hl=nl

In [ ]:
import smtplib, ssl
from email.message import EmailMessage

# Stel alle gegevens voor de SMTP-server in
port = 465  # For SSL
smtp_server = "smtp.gmail.com"
sender_email = "<jouw e-mailadres>"
receiver_email = "<e-mailadres van ontvanger>"
password = "<je paswoord>"

# Stel het bericht op
msg = EmailMessage()
msg.set_content("Hoi\nHierbij de foto!")
msg['Subject'] = "Hello from me!"
msg['From'] = sender_email
msg['To'] = receiver_email

# Voeg afbeelding toe als attachment
with open('images/transparent.png', 'rb') as f:
    file_data = f.read()
    msg.add_attachment(file_data, maintype='image', subtype='png', filename='picture.png')

# Verstuur e-mail
context = ssl.create_default_context()
with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, password)
    server.send_message(msg, from_addr=sender_email, to_addrs=receiver_email)
